In [1]:
import pandas as pd
import os
data_folder = "../data"

In [27]:
census = pd.read_csv(os.path.join(data_folder, 'census', 'hex_census.csv'))
runs = pd.read_csv(os.path.join(data_folder, 'run_reference.csv'))
census.columns

Index(['id', 'pop_2016', 'age_total', 'age_0_14', 'age_65_plus', 'lone_total',
       'lang_total', 'lang_french', 'lang_neither', 'inc_total', 'linc_lim',
       'linc_lico', 'imm_total', 'imm_imm', 'imm_recent', 'ind_total',
       'ind_ind', 'house_total', 'house_30p_shelter', 'tenant_total',
       'tenant_30p_shelter', 'lab_total', 'lab_inforce', 'lab_unemp'],
      dtype='object')

In [40]:
out = {
    'matrix_id': [],
    'pop_2016': [],
    'imm_recent': [],
    'age_0_14': [],
    'age_65_plus': [],
    'linc_lico': []
}
for idx, row in runs.iterrows():
    result = pd.read_csv(os.path.join(data_folder, 'results', f"{row['matrix_id']}_results.csv"))
    # Join to Census
    result = pd.merge(result, census, on='id')
    # Let's do population
    result['pop_2016_frac'] = result['pop_2016']/result['pop_2016'].sum()
    result['pop_frac'] = result['pop_2016_frac'] * result['population']
    
    result['imm_recent_frac'] = result['imm_recent']/result['imm_recent'].sum()
    result['imm_recent_pop'] = result['imm_recent_frac'] * result['population']
    
    result['age_0_14_frac'] = result['age_0_14']/result['age_0_14'].sum()
    result['age_0_14_pop'] = result['age_0_14_frac'] * result['population']
    
    result['age_0_14_frac'] = result['age_0_14']/result['age_0_14'].sum()
    result['age_0_14_pop'] = result['age_0_14_frac'] * result['population']
    
    result['age_65_plus_frac'] = result['age_65_plus']/result['age_65_plus'].sum()
    result['age_65_plus_pop'] = result['age_65_plus_frac'] * result['population']
    
    result['linc_lico_frac'] = result['linc_lico']/result['linc_lico'].sum()
    result['linc_lico_pop'] = result['linc_lico_frac'] * result['population']
    
    out['matrix_id'].append(row['matrix_id'])
    out['pop_2016'].append(result.pop_frac.sum())
    out['imm_recent'].append(result.imm_recent_pop.sum())
    out['age_0_14'].append(result.age_0_14_pop.sum())
    out['age_65_plus'].append(result.age_65_plus_pop.sum())
    out['linc_lico'].append(result.linc_lico_pop.sum())
    
    
summary = pd.DataFrame(out)

In [46]:
import altair as alt

summary_ref = pd.merge(summary, runs, on='matrix_id')
wda = summary_ref[summary_ref.period == 'WDP'][['week_of', 'pop_2016', 'imm_recent', 'age_0_14', 'age_65_plus', 'linc_lico']]
alt.Chart(wda.melt(id_vars=['week_of'])).mark_line().encode(
    alt.X('week_of:T'),
    alt.Y('value:Q', title="Persons Reachable in 45 Minutes"),
    alt.Color('variable:N')
).interactive()

alt.Chart(...)